In [1]:
import pandas as pd
import re

In [2]:
sds = pd.read_csv('../data/sds_cleaned.csv')

In [3]:
sds.head(2)

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko
1,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is session number two with machin...,2,"Sep 14, 2016",2016,Wednesday,Kirill Eremenko


In [4]:
sds['episode_date'] = pd.to_datetime(sds['episode_date'], format='%b %d, %Y' )

In [5]:
sds['episode_year'] = sds['episode_date'].dt.year

In [6]:
sds

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko
1,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is session number two with machin...,2,2016-09-14,2016,Wednesday,Kirill Eremenko
2,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number three with Nan...,3,2016-09-25,2016,Sunday,Kirill Eremenko
3,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode four with business str...,4,2016-10-02,2016,Sunday,Kirill Eremenko
4,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number five with fore...,5,2016-10-09,2016,Sunday,Kirill Eremenko
...,...,...,...,...,...,...,...,...,...,...,...
677,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:00This is episode number 679...,679,2023-05-16,2023,Tuesday,Jon Krohn
678,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:06\tThis is episode number 680 ...,680,2023-05-19,2023,Friday,Jon Krohn
679,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:00\tThis is episode number 6...,681,2023-05-23,2023,Tuesday,Jon Krohn
680,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:05\tThis is episode number 682 ...,682,2023-05-26,2023,Friday,Jon Krohn


In [7]:
 topics = [' SEOI PEI – The skin of water listen now',
           ' “What are you passionate about?”', 
           ' Sweat every day',
           ' Pride and humility',
           ' Compartmentalization', 
           ' Get out there!',
           ' Failure', 
           ' Happiness and problem solving',
           ' Get in touch',
           ' Secret to success', 
            ' What is reality?',
           ' Push yourself',
            ' Teamwork', 
            ' Getting things done',
           ' Selective ignorance',
            ' Date with destiny', 
            ' Conferences',
           ' Maker’s schedule vs. Manager’s schedule', 
            ' Dreams vs. Goals',
           ' My best tool',
            ' The best ideas', 
            ' Stimulate your creativity',
           ' The Quant Crunch', 
            ' Connecting the dots',
           ' The five balls of life', 
            ' Do it for yourself',
           ' Breaking patterns', 
            ' Your questions', 
       ' Why i became Vegetarian', 
        ' Computer vision', 
        ' Fermi questions',
       ' Do what you want', 
        ' Exponential Thinking', 
        ' The power of now',
        ' Willpower', 
        ' 100 Episodes',
        ' Board Games',
        ' Working Remotely',
       ' AlphaGo Zero', 
        ' How to Win Friends & Influence People',
       ' Expand Your Comfort Zone',
        ' The Power of Gratitude',
       ' 1-on-1 with Kirill', 
        ' New Year Resolutions',
       ' Technological Singularity', 
        ' Who Moved My Cheese?',
       ' Reckless Commitment', 
        ' Ender’s Game', 
       ' Instant Gratification Monkey',
       ' How do Lobsters Grow?', 
        ' Learn Blockchain!',
        ' Zone of Genius',
       ' Upper Limiting', 
        ' Coworking',
       ' Two Things to Remember and Two Things to Forget',
       ' Empathy and Compassion',
        ' The Trolley Problem',
       ' Have a Mentor', 
       ' Why Socializing is Vital',
       " Parkinson's Law to increase Productivity",
       ' Maximize Life with the Rapid Planning Method',
       ' Up Your Presentation Skills with Toastmasters',
       ' What I Learned from a 10 Day Detox',
       ' Love Languages and How They Impact Your Life',
       ' Should You Be Effective or Efficient?',
       ' Focus on Your Strengths and Ignore Your Weaknesses',
       ' Tips for a Bumpy Ride', 
        ' Why You Need to Go Beyond the Data',
       ' A Technology Detox Challenge',
       ' The Amazing Idea Behind Essentialism',  
       ' Why You Need Domain Knowledge in Data Science',
       ' Why Execution Trumps Knowledge',
       ' 6 Ways to Fill the Data Science Gap', 
        ' See You in San Diego',
       ' Ace the “Greatest Weakness” Interview Question',
       ' Why I Became Vegan', 
       ' Two Millimeter Shifts',
       ' What’s the Future of Data Science? Speaker Mashup Edition',
       ' Ideas and Execution', 
        ' Set Your Goals Higher',
        ' Re-live DSGO2018',
       ' Compete With Yourself', 
       ' What Is Amazing In Your Life', 
       ' Start A Great Day', 
        ' Data Science in Retail',
       ' 2018 in Numbers', 
        ' Hacks for reading more books',
       ' Flat Tyres Happen', 
        ' Sleep on it',
       ' How to Deal with Negative Emotions',
        ' Meditation',
       ' Guilt vs Shame', 
        ' Two Wolves', 
        ' Eating S.L.O.W.L.Y.',
       ' You Cannot Make Progress Without a Routine',
       ' Exploration vs Exploitation', 
        ' The Cold is My Master',
       ' The Six Months Rule', 
        ' Legacy', 
       ' Pura Vida', 
        ' Your Tribe', 
        ' Make It About Yourself', 
        ' Amazing',
       ' Who Inspires You?', 
        ' Look for the Horse',
        ' Good!',
       ' Go through the Motions',
        ' No Coaching', 
        ' Better Than Perfect',
       ' Too Many Photos',
       ' My Top 5 Productivity Hacks',
        ' Coronavirus',
       ' Negative Coefficients', 
        ' Racism and Discrimination',
       ' Importance of Sleep', 
        ' Define Your Own Success',
       ' Future-Proof Your Career', 
       ' Remember to Wind Down',
       ' Expose Yourself to New Ideas Regularly',
       ' Use Your Unconscious Mind', 
       ' Get a Headhunter', 
        ' Perception vs. Emotion',
       ' Start Your Own Morning Ritual', 
        ' Teach It',
       ' Five Job Hunting Tips', 
        ' Emotional Burnout', ' Think Bigger',
       ' Face Your Demons', 
        ' The Narrative Arc in Storytelling',
       ' Abandon Hope', 
        ' Meaning is Everything',
       ' Communicate Your Needs', 
        ' Needs vs. Wants',
       ' My Advice for Career Success', 
        ' Play With Feeling',
       ' Wheel of Life', 
        ' Pain vs. Suffering', 
        ' The Shift',
       ' The Internal Conflict Model', 
        ' Intellect and Intelligence',
       ' One-on-one with Kirill',
        ' Hello from Jon and Welcome to 2021',
       ' Attention Sharpening Tools Part 1',
       ' Attention Sharpening Tools Part 2',
       ' The Staggering Pace of Progress', 
        ' The Pomodoro Technique',
       ' Behind the Scenes', 
        ' The History of Algebra',
       ' It Could Be Even Better', 
        ' My Favorite Books',
       ' The Learning Never Stops (so Relax)',
        ' Peer-Driven Learning',
       ' Five Keys to Success', 
        ' Top Five Resume Tips',
       ' The Continuous Calendar',
        ' The History of Calculus',
       ' The Price of Your Attention',
       ' The World is Awful (and it’s Never Been Better)',
       ' How to Instantly Appreciate Being Alive',
        ' 2040',
       ' How Only Beginners Know Everything',
       ' Managing Imposter Syndrome', 
        ' Building Your Ant Hill',
       ' Does Caffeine Hurt Productivity? (Part 1)',
       ' Does Caffeine Hurt Productivity? (Part 2',
       ' Does Caffeine Hurt Productivity? (Part 3', 
        ' Fail More',
       ' The Normal Anxiety of Content Creation',
       ' Mutable vs Immutable Conditions', 
        ' A Holiday Greeting',
       ' What I Learned in 2021', 
        ' Daily Habit #1', ' Daily Habit #2',
       ' Continuous Calendar for 2022', 
        ' Daily Habit #3',
       ' Daily Habit #4', 
        ' Daily Habit #5',
        ' Daily Habit #6',
       ' The Most Popular SuperDataScience Episodes of 2021',
       ' Daily Habit #7', 
        ' The Best Time to Plant a Tree',
       ' Daily Habit #9', 
        ' Music for Deep Work', 
        ' Tech Startup Dramas',
        ' Daily Habit #10',
       ' We Are Living in Ancient Times',
        ' Ignition',
       ' Four Thousand Weeks', 
        ' Who Dares Wins',
       ' More Guests on Fridays',
        ' What I Learned in 2022',
       ' The Most Popular SuperDataScience Podcast Episodes of 2022',
          ' Career choice, disruptions in finance and application stacks',
 ' How Constant Learning Created a Jet-Set Career',
 ' Tips to Improve your Memory',
 ' How to be Happy and Successful',
 ' Boost Your Self-Confidence',
 ' Ask the Right Question',
 ' Your Core Strength',
 ' Learning Something New',
 ' Solitude Deprivation',
 ' Love Yourself',
 ' The Passion Paradox',
 ' Who You Become',
 ' Trial by Fire',
 ' Contemplation',
 ' Meet the Team',
 ' Mentorship',
 ' Diets',
 ' Proximity is Power #2',
 ' Many Ways to Fail & Five Ways to Succeed in Startups',
 ' The Power of Coaching',
 ' Emotions, Relationships, and Being Kind During the Pandemic',
 ' Depression and Suicidal Thoughts',
 ' The Power of Women in STEM',
 ' Maelstrom, Chaos, and Mayhem',
 ' 10 Tips to Become a Master Presenter',
 ' Stand More - Sit Less',
 ' Real-World Applications of Digital Twins',
 ' The End of Jobs',
 ' Yoga Nidra',
 ' Open-Access Publishing',
 ' Yoga Nidra Practice with Steve Fazzari',
 ' Daily Habit #11',
 ' Thriving on Information Overload',
 ' The Four Requirements for Expertise (beyond the “10,000 Hours”)',
 ' The Joy of Atelic Activities',
 ' Burnout']

In [8]:
# function to create dataframe with non datascience episodes
def subset_by_episode_name(df, episode_names):
    return df[df['episode_name'].isin(episode_names)]

In [9]:
non_datascience = subset_by_episode_name(sds, topics).reset_index(drop = True)

In [10]:
non_datascience.to_csv('../data/non_datascience.csv', index = False)

## Creating datascience episode dataframe

In [11]:
# Function to leave datascience episodes in sds dataframe
def remove_by_episode_name(df, episode_names):
    return df[~df['episode_name'].isin(episode_names)]

In [12]:
sds_datascience = remove_by_episode_name(sds, topics).reset_index(drop = True)

In [13]:
sds_datascience.head()

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko
1,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is session number two with machin...,2,2016-09-14,2016,Wednesday,Kirill Eremenko
2,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number three with Nan...,3,2016-09-25,2016,Sunday,Kirill Eremenko
3,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode four with business str...,4,2016-10-02,2016,Sunday,Kirill Eremenko
4,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number five with fore...,5,2016-10-09,2016,Sunday,Kirill Eremenko


In [14]:
sds_datascience.to_csv('../data/sds_datascience.csv', index = False)

In [15]:
sds_datascience['context_episode'].nunique()

91

In [16]:
sds_datascience = pd.read_csv('../data/sds_datascience.csv')

In [17]:
sds_datascience.head(2)

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko
1,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is session number two with machin...,2,2016-09-14,2016,Wednesday,Kirill Eremenko


In [18]:
# Duplicating context_episode column
sds_datascience['class'] = sds_datascience['context_episode']

In [19]:
sds_datascience

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode,class
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Business Data Science Database
1,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is session number two with machin...,2,2016-09-14,2016,Wednesday,Kirill Eremenko,Machine Learning Data Science
2,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number three with Nan...,3,2016-09-25,2016,Sunday,Kirill Eremenko,Machine Learning R Programming Data Science
3,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode four with business str...,4,2016-10-02,2016,Sunday,Kirill Eremenko,Business Data Science
4,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill:\tThis is episode number five with fore...,5,2016-10-09,2016,Sunday,Kirill Eremenko,Data Science
...,...,...,...,...,...,...,...,...,...,...,...,...
446,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:00This is episode number 679...,679,2023-05-16,2023,Tuesday,Jon Krohn,Business Data Science Artificial Intelligence
447,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:06\tThis is episode number 680 ...,680,2023-05-19,2023,Friday,Jon Krohn,Business Data Science
448,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:00\tThis is episode number 6...,681,2023-05-23,2023,Tuesday,Jon Krohn,Machine Learning Data Science Python
449,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:05\tThis is episode number 682 ...,682,2023-05-26,2023,Friday,Jon Krohn,Business Data Science


In [20]:
Data_Visualization = ['Data Science Database Data Visualization',
                      'Tableau Data Visualization',
                      'Business Data Visualization',
                      'Data Science Data Visualization',
                      'Data Science Tableau Excel Data Visualization',
                      'Data Science Tableau Data Visualization',
                      'Tableau Database Data Visualization',
                      'Business Data Science Tableau Data Visualization',
                      'Business Tableau Data Visualization',
                      'Data Science Excel Data Visualization',
                      'Business Data Science Data Visualization']

In [21]:
sds_datascience['class'] = sds_datascience['class'].replace(Data_Visualization, 'Data Visualization')

In [22]:
Programming = ['Excel', 
               'Python',
               'Business Data Science Excel',
               'R Programming Data Science Python',
               'Business R Programming',
               'R Programming Python', 
               'Business R Programming Data Science',
               'R Programming Database', 
               'Data Science Python',
               'R Programming Data Science',
               'R Programming Data Science Excel']

In [23]:
sds_datascience['class'] = sds_datascience['class'].replace(Programming, 'Programming')

In [24]:
Programming_Data_Visualization = ['Business R Programming Tableau Data Visualization',
                             'R Programming Data Science Data Visualization',
                             'R Programming Tableau Database Data Visualization',
                             'Business R Programming Data Science Python Database Data Visualization']

In [25]:
sds_datascience['class'] = sds_datascience['class'].replace(Programming_Data_Visualization, 'Programming, Data Visualization')

In [26]:
Database = ['Business Data Science Database', 
            'Data Science Python Database',
            'Business Python Database',
            'Database',
            'Business Database', 
            'Data Science Database']

In [27]:
sds_datascience['class'] = sds_datascience['class'].replace(Database, 'Database')

In [28]:
Machine_Learning = ['Machine Learning Data Science',
                     'Machine Learning R Programming Data Science',
                     'Machine Learning', 
                     'Business Machine Learning R Programming Data Science Python Excel',
                     'Machine Learning R Programming Data Science Excel Database',
                     'Business Machine Learning Python Database',
                     'Machine Learning Data Science Python',
                     'Business Machine Learning Data Science Python',
                     'Business Machine Learning Tableau Data Visualization',
                     'Business Machine Learning Data Science', 
                     'Machine Learning Data Science Database',
                     'Business Machine Learning', 
                     'Machine Learning Python Database',
                     'Machine Learning Data Visualization',
                     'Machine Learning Data Science Productivity',
                     'Business Machine Learning R Programming Data Science']

In [29]:
sds_datascience['class'] = sds_datascience['class'].replace(Machine_Learning, 'Machine Learning')

In [30]:
Machine_Learning_AI = ['Business Machine Learning Data Science Artificial Intelligence',
                        'Machine Learning Data Science Artificial Intelligence Blockchain',
                        'Business Machine Learning Data Science Artificial Intelligence Database',
                        'Machine Learning Data Science Artificial Intelligence']

In [31]:
sds_datascience['class'] = sds_datascience['class'].replace(Machine_Learning_AI, 'Machine Learning, Artificial Intelligence')

In [32]:
Machine_Learning_AI_Deep_Learning = ['Business Machine Learning Data Science Python Artificial Intelligence Deep Learning',
                 'Business Machine Learning Data Science Artificial Intelligence Deep Learning',
                 'Machine Learning Data Science Python Artificial Intelligence Database Deep Learning',
                 'Machine Learning Data Science Artificial Intelligence Deep Learning',
                 'Machine Learning Data Science Artificial Intelligence Database Blockchain Deep Learning',
                 'Business Machine Learning R Programming Data Science Artificial Intelligence Deep Learning',
                 'Machine Learning Artificial Intelligence Blockchain Deep Learning',
                 'Machine Learning Data Science Artificial Intelligence Blockchain Deep Learning',
                 'Machine Learning Artificial Intelligence Deep Learning']

In [33]:
sds_datascience['class'] = sds_datascience['class'].replace(Machine_Learning_AI_Deep_Learning, 'Machine Learning, Artificial Intelligence, Deep Learning')

In [34]:
AI = ['Artificial Intelligence', 
        'Business Artificial Intelligence',
        'Business Data Science Artificial Intelligence',
        'Business Artificial Intelligence Database',
        'Data Science Artificial Intelligence']

In [35]:
sds_datascience['class'] = sds_datascience['class'].replace(AI, 'Artificial Intelligence')

In [36]:
Machine_Learning_Deep_Learning = ['Machine Learning Data Science Python Deep Learning',
                                    'Business Machine Learning Data Science Deep Learning',
                                    'Machine Learning Data Science Deep Learning',
                                    'Business Machine Learning R Programming Data Science Python Deep Learning',
                                    'Machine Learning Data Science Database Deep Learning',
                                    'Data Science Deep Learning']

In [37]:
sds_datascience['class'] = sds_datascience['class'].replace(Machine_Learning_Deep_Learning, 'Machine Learning, Deep Learning')

In [38]:
AI_Deep_Learning = ['Data Science Artificial Intelligence Deep Learning',
                       'Business Data Science Artificial Intelligence Deep Learning']

In [39]:
sds_datascience['class'] = sds_datascience['class'].replace(AI_Deep_Learning, 'Artificial Intelligence, Deep Learning')

In [40]:
Data_Science = ['Business Data Science', 
                  'Data Science', 
                  'Uncategorized',
                  'Business', 
                  'Data Science Statistics', 
                  'Life Philosophy Data Science']

In [41]:
sds_datascience['class'] = sds_datascience['class'].replace(Data_Science, 'Data Science')

In [42]:
Blockchain = ['Data Science Blockchain',
              'Database Blockchain',
              'Blockchain']

In [43]:
sds_datascience['class'] = sds_datascience['class'].replace(Blockchain, 'Blockchain')

In [44]:

Data_Science_Productivity = ['Data Science Python Productivity',
                            'Data Science Productivity']

In [45]:
sds_datascience['class'] = sds_datascience['class'].replace(Data_Science_Productivity, 'Data Science, Productivity')

In [46]:
Deep_Learning = ['Data Science Deep Learning', 
                 'Business Data Science Deep Learning']

In [47]:
sds_datascience['class'] = sds_datascience['class'].replace(Deep_Learning, 'Deep Learning')

In [48]:
career = ['Data Science Career Tips',
          'Business Data Science Career Tips',
          'Data Science Python Career Tips',
          'Business Data Science Artificial Intelligence Career Tips',
          'Machine Learning Data Science Career Tips']

In [49]:
sds_datascience['class'] = sds_datascience['class'].replace(career, 'Data Science, Career')

In [50]:
sds_datascience['class'].unique()

array(['Database', 'Machine Learning', 'Data Science', 'Programming',
       'Data Visualization', 'Artificial Intelligence',
       'Machine Learning, Artificial Intelligence, Deep Learning',
       'Programming, Data Visualization',
       'Artificial Intelligence, Deep Learning',
       'Machine Learning, Deep Learning',
       'Machine Learning, Artificial Intelligence', 'Blockchain',
       'Deep Learning', 'Data Science, Career',
       'Data Science, Productivity'], dtype=object)

In [51]:
sds_datascience.to_csv('../data/sds_datascience_class.csv', index = False)

In [52]:
def find_pattern(row):
    episode_split_text = []
    speaker = []
    episode_no = str(row['episode_number'])
    guest_name = str(row['guest_name'].strip())
    host_episode = str(row['host_episode'].strip())
    text = str(row['text_episode'])

    # Different patterns of guest name and host name present in text
    guest_first_name_1 = guest_name.split()[0]
    guest_first_name_2 = guest_name.split()[1] if len(guest_name.split()) > 1 else '' # if any credentials are present in fron of name like Dr.
    host_first_name = host_episode.split()[0]
    guest_last_name = guest_name.strip().split()[-1]
    guest_first_middle_name = ' '.join(guest_name.split()[:2])
    guest_middle_last_name = ' '.join(guest_name.split()[1:3])
    guest_first_last_name = " ".join([guest_first_name_1, guest_last_name])
    first_guest_name = ' '.join(guest_name.split()[:2])
    second_guest_first_name = guest_name.strip().split()[-2] if len(guest_name.split()) > 1 else ''
    second_guest_full_name = " ".join([second_guest_first_name, guest_last_name]) if second_guest_first_name else ''
   
    # Conditions to look in text
    pattern = f"{guest_name}:|{host_episode}:|{guest_first_name_1}:|{guest_first_name_2}:|{host_first_name}:|{guest_last_name}:|{guest_first_middle_name}:|{guest_middle_last_name}:|{guest_first_last_name}:|{first_guest_name}:|{second_guest_full_name}:"
    
    # Find all occurrences of the pattern and their corresponding positions
    pattern_matches = re.finditer(pattern, text)
    
    prev_end = 0
    for match in pattern_matches:
        start = match.start()
        end = match.end()
        
        # Add the text between the previous and current pattern occurrence to the split_text list
        episode_split_text.append(text[prev_end:start].strip())
        
        prev_end = end
        
        # Add the speaker information for the previous split text
        speaker.append(text[start:end-1].strip())
    
    # Add the remaining text after the last pattern occurrence to the split_text list
    episode_split_text.append(text[prev_end:].strip())
    
    # Add the speaker information for the remaining split text
    speaker.append('')
    
    # Shift the index of the speaker list by 1
    speaker = [''] + speaker[:-1]

    # Create a DataFrame from the split text and speaker
    text_df = pd.DataFrame({'episode_number': episode_no, 'episode_split_text': episode_split_text, 'speaker': speaker})
    
    return text_df

### Splitting the text datascience episode dataframe

In [53]:
# Loop to process each row of sds_datascience dataframe
sds_ds_split = pd.DataFrame(columns=['episode_number', 'episode_split_text', 'speaker'])
for i, row in sds_datascience.iterrows():
    split_text = find_pattern(row)
    sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sds_ds_split = sds_ds_split.append(split_text, ignore_index = False)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/1400762108.py:5: FutureWarning: The frame.append method is deprecated and will be

In [54]:
sds_ds_split['episode_split_text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0).value_counts()

False    49448
True       359
Name: episode_split_text, dtype: int64

In [55]:
sds_ds_text = sds_ds_split[sds_ds_split['episode_split_text'] != '']

In [56]:
sds_ds_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49448 entries, 1 to 89
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   episode_number      49448 non-null  object
 1   episode_split_text  49448 non-null  object
 2   speaker             49448 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [57]:
sds_ds_text

,episode_number,episode_split_text,speaker
1,1,This is episode number one with ex-chemical en...,Kirill
2,1,"Hey guys, welcome to the Podcast. I’ve got Rub...",Kirill
3,1,Thank you! Thanks for having me over. I’m doin...,Ruben
4,1,Awesome. It’s great to hear you and for those ...,Kirill
5,1,"Yeah sure. So, I’m the senior manager of analy...",Ruben
...,...,...,...
85,683,"01:17:22Yeah, right. Yeah, as I mean, it actua...",Jon Krohn
86,683,01:17:45\tHappy to.,Matar Haller
87,683,"01:17:46\tNice. Well, yeah, so you mentioned p...",Jon Krohn
88,683,01:18:04\tThank you for having me. This was fa...,Matar Haller


In [58]:
sds_ds_text['episode_number'] = sds_ds_text['episode_number'].astype('int')

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/4240092265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sds_ds_text['episode_number'] = sds_ds_text['episode_number'].astype('int')


In [59]:
sds_ds_text = pd.merge(sds_datascience, sds_ds_text, on = 'episode_number')

In [60]:
sds_ds_text

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode,class,episode_split_text,speaker
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,This is episode number one with ex-chemical en...,Kirill
1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Hey guys, welcome to the Podcast. I’ve got Rub...",Kirill
2,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Thank you! Thanks for having me over. I’m doin...,Ruben
3,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Awesome. It’s great to hear you and for those ...,Kirill
4,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Yeah sure. So, I’m the senior manager of analy...",Ruben
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49443,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"01:17:22Yeah, right. Yeah, as I mean, it actua...",Jon Krohn
49444,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,01:17:45\tHappy to.,Matar Haller
49445,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"01:17:46\tNice. Well, yeah, so you mentioned p...",Jon Krohn
49446,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,01:18:04\tThank you for having me. This was fa...,Matar Haller


In [61]:
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\\t|\\n|\\xa0', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(?\d{2}:\d{2}\):| ?(\d{2}:)+\d{2}', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(background music plays\)', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(background music\)', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\[inaudible\]', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(inaudible\)', '')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("That\\'s", ' That is')
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("it\\'s", "it is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("They\\'re", " They are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("they\\'re", "they are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("that\\'s", "that is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("you\\'re", "you are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("You\\'re", " You are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("I\\'m", " I am")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("here\\'s", "here is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("You\'re", "You are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("you\'re", "you are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("we\'re", "we are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("I\'ve", "I have")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("I\'ll", "I will")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("He\'s", "He is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("he\'s", "he is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("what\'s", "what is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("here\\'s", "here is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("there\\'s", "there is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("isn\\'t", "is not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("didn\\'t", "did not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("Yeah, yeah, yeah, yeah", " Yes")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("Yeah", " Yes")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("yeah", "yes")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("don\\'t", "do not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("’", "'")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("haven\'t", "have not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("doesn\'t", "does not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("we\'ve", "we have")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("we\'d", "we had")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("It\'s", " It is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("can\'t", "cannot")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("who\'s", "whose")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("wouldn\'t", "would not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("there\'re", "there are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("There\'re", " There are")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("it\'s", "it is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("What\'s", " What is")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("Right, right, right", "Right")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("you\'ll", "you will")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("there\'ll", "there will")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("let\'s", "let us")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("Yes, yes, yes", "yes")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("would\'ve", "would have")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("AI", "artificial intelligence")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("what\\'ll", "what will")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("won\'t", "would not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("shouldn\'t", "should not")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("\[crosstalk\]", "")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("A.I.", "artificial intelligence")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("Five-Minute", "Five Minute")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("SuperDataScience", "")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("I\'d", "I had")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("They\'ve", "They Have")
sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace("I\'m", "I am")

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/3019695698.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\\t|\\n|\\xa0', '')
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/3019695698.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(?\d{2}:\d{2}\):| ?(\d{2}:)+\d{2}', '')
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/3019695698.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  sds_ds_text['episode_split_text'] = sds_ds_text['episode_split_text'].str.replace('\(background music plays\)', '')
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_1766/3019695698.py:4: FutureWarning: The default value of regex wil

In [62]:
sds_ds_text

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode,class,episode_split_text,speaker
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,This is episode number one with ex-chemical en...,Kirill
1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Hey guys, welcome to the Podcast. I've got Rub...",Kirill
2,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Thank you! Thanks for having me over. I'm doin...,Ruben
3,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Awesome. It is great to hear you and for thos...,Kirill
4,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Yes sure. So, I'm the senior manager of analy...",Ruben
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49443,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"Yes, right. Yes, as I mean, it actually, it ...",Jon Krohn
49444,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,Happy to.,Matar Haller
49445,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"Nice. Well, yes, so you mentioned potentially ...",Jon Krohn
49446,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,Thank you for having me. This was fascinating ...,Matar Haller


In [63]:
sds_ds_text.to_csv('../data/sds_ds_text.csv', index = False)

### processing nondatascience episode dataframe

In [ ]:
# Loop to process each row of non_datascience dataframe
sds_nds_split = pd.DataFrame(columns=['episode_number', 'episode_split_text', 'speaker'])
for i, row in non_datascience.iterrows():
    split_text = find_pattern(row)
    sds_nds_split = sds_nds_split.append(split_text, ignore_index = False)

In [ ]:
sds_nds_split

In [ ]:
sds_nds_split['episode_split_text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0).value_counts()

In [ ]:
sds_nds_split['episode_number'].value_counts()

In [ ]:
sds_nds_text = sds_nds_split[sds_nds_split['episode_split_text'] != '']

In [ ]:
sds_nds_text.info()

In [ ]:
sds_nds_text['episode_number'] = sds_nds_text['episode_number'].astype('int')

In [ ]:
sds_nds_text = pd.merge(non_datascience, sds_nds_text, on = 'episode_number')

In [ ]:
sds_nds_text.head()

In [ ]:
sds_nds_text['episode_split_text'] = sds_nds_text['episode_split_text'].str.replace('\\t|\\n|\\xa0', '')
sds_nds_text['episode_split_text'] = sds_nds_text['episode_split_text'].str.replace('\(?\d{2}:\d{2}\):| ?(\d{2}:)+\d{2}', '')
sds_nds_text['episode_split_text'] = sds_nds_text['episode_split_text'].str.replace('\(background music plays\)', '')

In [ ]:
sds_nds_text

In [ ]:
sds_nds_text.to_csv('../data/sds_nds_text.csv', index = False)

In [ ]:
import spacy
from spacy import displacy
from collections import Counter

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
 def extract_information(row):
    guest_name = row['guest_name'])
    guest_info = str(row['guest_info'])
    
    doc = nlp(guest_info)
    
    for sent in doc.sents:
        for token in sent:
            if token.text == guest_name and token.dep_ == 'nsubj':
                verb = [child for child in token.head.children if child.dep_ == 'ROOT']
                if verb:
                    return verb[0].text
    return None

In [ ]:
row = sds.loc[0]

In [ ]:
extract_information(row)